# Import Data CSV ke PostgreSQL dengan Python

**Nama:** Hafiz Alfariz

Notebook ini berisi langkah-langkah untuk mengimpor data dari file CSV ke database PostgreSQL.

**Konfigurasi koneksi:**
- Host: localhost
- Port: 5434
- Database: airflow
- Username: airflow
- Password: airflow
- Tabel: table_m3
- File CSV: P2M3_hafiz_alfariz_data_raw.csv

# 1. Import Library

In [1]:
# Import library utama
import pandas as pd
import psycopg2

# Import utility untuk proses data
from io import StringIO

# 2. Load data.

In [2]:
# Baca data dari file CSV (data raw milestone)
csv_file = 'P2M3_hafiz_alfariz_data_raw.csv'
df = pd.read_csv(csv_file)
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3


# 3. Membuat koneksi PostgreSQL

In [3]:
# Konfigurasi koneksi PostgreSQL dan parameter tabel
server = 'postgres_m3'  # Nama server
host = 'localhost'
port = 5434
database = 'airflow'  # Maintenance database
user = 'airflow'
password = 'airflow'
table_name = 'table_m3'

# Membuat koneksi ke PostgreSQL 
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=database,
    user=user,
    password=password
)
cursor = conn.cursor()

# 4. Membuat Tabel

In [4]:
# DDL raw data
ddl = '''
CREATE TABLE IF NOT EXISTS table_m3 (
    "Invoice ID" VARCHAR(20),
    "Branch" VARCHAR(10),
    "City" VARCHAR(30),
    "Customer type" VARCHAR(20),
    "Gender" VARCHAR(10),
    "Product line" VARCHAR(50),
    "Unit price" NUMERIC(10,2),
    "Quantity" INTEGER,
    "Tax 5%" NUMERIC(12,2),
    "Sales" NUMERIC(12,2),
    "Date" DATE,
    "Time" TIME,
    "Payment" VARCHAR(20),
    "cogs" NUMERIC(12,2),
    "gross margin percentage" NUMERIC(8,2),
    "gross income" NUMERIC(12,2),
    "Rating" NUMERIC(5,2)
);
'''
cursor.execute(ddl)
conn.commit()

# 5. Import Raw Data

In [5]:
# Konversi DataFrame ke CSV string tanpa header dan index
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, header=False)
csv_buffer.seek(0)

# Hapus data lama agar tidak duplikat jika import ulang
cursor.execute(f"TRUNCATE TABLE {table_name};")
conn.commit()

# Query untuk import data
copy_sql = f"""
COPY {table_name} FROM STDIN WITH CSV
"""
conn.rollback()
cursor.copy_expert(copy_sql, csv_buffer)
conn.commit()

# 6. Verifikasi

In [6]:
# Verifikasi hasil import data
cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
row_count = cursor.fetchone()[0]
print(f"Jumlah baris di tabel {table_name}: {row_count}")

cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
for row in cursor.fetchall():
    print(row)

Jumlah baris di tabel table_m3: 1000
('750-67-8428', 'Alex', 'Yangon', 'Member', 'Female', 'Health and beauty', Decimal('74.69'), 7, Decimal('26.14'), Decimal('548.97'), datetime.date(2019, 1, 5), datetime.time(13, 8), 'Ewallet', Decimal('522.83'), Decimal('4.76'), Decimal('26.14'), Decimal('9.10'))
('226-31-3081', 'Giza', 'Naypyitaw', 'Normal', 'Female', 'Electronic accessories', Decimal('15.28'), 5, Decimal('3.82'), Decimal('80.22'), datetime.date(2019, 3, 8), datetime.time(10, 29), 'Cash', Decimal('76.40'), Decimal('4.76'), Decimal('3.82'), Decimal('9.60'))
('631-41-3108', 'Alex', 'Yangon', 'Normal', 'Female', 'Home and lifestyle', Decimal('46.33'), 7, Decimal('16.22'), Decimal('340.53'), datetime.date(2019, 3, 3), datetime.time(13, 23), 'Credit card', Decimal('324.31'), Decimal('4.76'), Decimal('16.22'), Decimal('7.40'))
('123-19-1176', 'Alex', 'Yangon', 'Member', 'Female', 'Health and beauty', Decimal('58.22'), 8, Decimal('23.29'), Decimal('489.05'), datetime.date(2019, 1, 27), da